<a href="https://colab.research.google.com/github/pbevan1/Word-Level-Language-Model-Poirot/blob/main/poirot_recurrent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**The raw text version of 'Poirot Investigates' by Agatha Christie is downloaded from Project Guttenberg**

In [137]:
import numpy as np
import string
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.utils import to_categorical
import itertools

In [1]:
!curl -O https://www.gutenberg.org/files/61262/61262-0.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  326k  100  326k    0     0   280k      0  0:00:01  0:00:01 --:--:--  280k


**Unwanted text (publishing information etc) is deleted from the text and a new file saved as 'Poirot.txt'**

In [9]:
#deleting unwanted text and front and back of book
with open('61262-0.txt') as old, open('Poirot.txt', 'w') as new:
    lines = old.readlines()
    new.writelines(lines[110:-374])

In [14]:
#opening the text file, saving into memory as `Poirot` and closing the file
text = open('Poirot.txt', 'r')
Poirot = text.read()
text.close()

#printing first 1000 characters to double check
print(Poirot[:1000])

  The Adventure of “The Western Star”

I was standing at the window of Poirot’s rooms looking out idly on
the street below.

“That’s queer,” I ejaculated suddenly beneath my breath.

“What is, _mon ami_?” asked Poirot placidly, from the depths of
his comfortable chair.

“Deduce, Poirot, from the following facts! Here is a young lady,
richly dressed—fashionable hat, magnificent furs. She is coming
along slowly, looking up at the houses as she goes. Unknown to her,
she is being shadowed by three men and a middle-aged woman. They
have just been joined by an errand boy who points after the girl,
gesticulating as he does so. What drama is this being played? Is
the girl a crook, and are the shadowers detectives preparing to
arrest her? Or are _they_ the scoundrels, and are they plotting to
attack an innocent victim? What does the great detective say?”

“The great detective, _mon ami_, chooses, as ever, the simplest
course. He rises to see for himself.” And my friend joined me at
the window.


In [116]:
# using keras' `text_to_word_sequence` function to tokenise the text
# addition characters added to the filters to account for the different type
# of quotation characters found in the text

tokens = Poirot.split()
table = str.maketrans('', '', string.punctuation)
tokens = [w.translate(table) for w in tokens]
cleaned_tokens = [word for word in tokens if word.isalpha()]
cleaned_tokens = [word.lower() for word in tokens] 

#cleaned_tokens = text_to_word_sequence(stripped)
print(f'There are {len(cleaned_tokens)} words in the cleaned version of Poirot Investigates ({len(set(cleaned_tokens))} unique words).')
print('------------------------------------------------------------------------------------------------------')
print(f'Sample of 10 tokens: {cleaned_tokens[:10]}')

There are 47904 words in the cleaned version of Poirot Investigates (5594 unique words).
------------------------------------------------------------------------------------------------------
Sample of 10 tokens: ['the', 'adventure', 'of', 'western', 'i', 'was', 'standing', 'at', 'the', 'window']


In [131]:
length = 50 + 1
sequences = list()
for i in range(length, len(cleaned_tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print(f'Total Sequences: {len(sequences)}')

Total Sequences: 47853


In [132]:
txt = '\n'.join(sequences)
txt_lines = txt.split('\n')
# defining tokenizer class to encode word seuences
encoder = Tokenizer()
encoder.fit_on_texts(txt_lines)
encoded_lines = encoder.texts_to_sequences(txt_lines)
print('--first line of text shown below in encoded form--')
print(encoded_lines[0])

--first line of text shown below in encoded form--
[1, 1434, 4, 990, 6, 8, 587, 20, 1, 272, 4, 837, 355, 48, 2825, 23, 1, 586, 1173, 6, 989, 224, 988, 21, 734, 14, 140, 156, 15, 732, 31, 1, 5592, 4, 13, 2824, 422, 15, 31, 1, 446, 333, 105, 14, 3, 113, 115, 5589, 987, 585, 1903]


In [133]:
word_index = encoder.word_index
print('printing 10 values from the `word_index` dictionary')
print('---------------------------------------------------')
dict(itertools.islice(word_index.items(), 10))

printing 10 values from the `word_index` dictionary
---------------------------------------------------


{'a': 3,
 'and': 5,
 'he': 9,
 'i': 6,
 'in': 7,
 'of': 4,
 'that': 10,
 'the': 1,
 'to': 2,
 'was': 8}

In [158]:
word_dim = len(set(cleaned_tokens))+1
encoded_lines = np.array(encoded_lines)
X, y = encoded_lines[:,:-1], encoded_lines[:,-1]
y = to_categorical(y, num_classes=word_dim)
sequence_len = X.shape[1]

In [159]:
sequence_len

50

**The model is defined**

In [160]:
# recurrent model